In [240]:
import random
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [260]:
# ------------- * FUNCTIONS * -------------

def get_content(file_path):
    data = pd.read_csv(file_path)
    labels = data['Health_Issue']
    features = data.drop('Health_Issue', axis=1)
    
    # ..transforming categorical values into numeric values..
    for column in features.columns:
        if(features[column].dtype == 'object'):
            features[column] = features[column].astype('category').cat.codes
            
    testing_samples = int(20 * len(data) / 100)
    
    testing_labels = labels.iloc[:testing_samples].to_numpy()
    testing_features = features.iloc[:testing_samples].to_numpy()
    
    training_labels = labels.iloc[testing_samples:].to_numpy()
    training_features = features.iloc[testing_samples:].to_numpy()
    
    scaler = StandardScaler()
    # ..the scaler is used to normalize input data so
    # it all fit into the same scale, because, it there's numbers
    # that are too distinct, the bigger ones have more weight 
    # in the models result, and it leads to bigger errors..
            
    return testing_labels, scaler.fit_transform(testing_features), training_labels, scaler.fit_transform(training_features)

def initialize_weights(input_dim, output_dim):
    return np.random.randn(output_dim, input_dim) * np.sqrt(2.0 / (input_dim + output_dim))
    # ..the weights being too big or too small can
    # impact directly the learning process, in this case,
    # xavier initialization is applied because is a good
    # match for the tanh activation function..

def test_model():
    sample = random.randrange(0, testing_samples)
    
    input_biased = np.hstack((bias, testing_features[sample]))
        
    output1 = np.tanh(weights1.dot(input_biased))
    output1_biased = np.hstack((bias, output1))
    
    output2 = np.tanh(weights2.dot(output1_biased))
    output2_biased = np.hstack((bias, output2))
    
    output3 = np.tanh(weights3.dot(output2_biased))
    output3_biased = np.hstack((bias, output3))
    
    result = np.tanh(weights4.dot(output3_biased))
    
    return result, testing_labels[sample]

def early_stopping():
    testing_times = 100
    testing_errors = np.zeros(testing_times)
    
    for k in range(testing_times):
        test_result, test_label = test_model()
        error = test_label - test_result
        testing_errors[k] = (error * error)/2
        
    mean_error = testing_errors.mean()
        
    return True if mean_error < 0.05 else False
    # ..using samples the model didn't use for 
    # training to check it's performance..
    
def reached_convergence():
    if(len(errors_mean) < 500):
        return False
    
    diff = np.diff(errors_mean[-25:])
    result = np.all(diff)
    
    return True if result >= 0 else False
    # ..calculates if the lastest 25 values in 
    # errors array are almost the same, indicating 
    # they are not getting any lower...

def save_weights(filename):
    weights_dict = {
        'weights1': weights1,
        'weights2': weights2,
        'weights3': weights3,
        'weights4': weights4
    }
    
    np.save(filename, weights_dict) 
        
def load_weights(filename):
    if not os.path.exists(filename):
        raise FileNotFoundError(f"No weights file found at {filename}")
    
    weights = np.load(filename, allow_pickle=True)  
    return weights["'weights1'"], weights["'weights2'"], weights["'weights3'"], weights["'weights4'"]

In [256]:
# ------------- * CONSTANTS * -------------

dataset = 'synthetic_covid_impact_on_work.csv'
testing_labels, testing_features, training_labels, training_features = get_content(dataset)

training_samples = len(training_features)
testing_samples = len(testing_features)

epocs = 100000 
# ..each epoc represents the time when all
# the data has been ran throught, if it's too big
# it's probably going to lead your model to overfitting..

learning_rate = 0.05
# ..keep it low, this value has 
# a lot of power in progressing the weights..

patterns = training_features.shape[1]
# ..how many features there is 
# to train from..

bias = 1
# ..changes the function's angle..

input_neurons = patterns
hidden_neurons1 = 82
hidden_neurons2 = 128
hidden_neurons3 = 64
output_neurons = 1

In [257]:

# ------------- * VARIABLES * -------------
weights1 = initialize_weights(input_neurons + 1, hidden_neurons1)
weights2 = initialize_weights(hidden_neurons1 + 1, hidden_neurons2)
weights3 = initialize_weights(hidden_neurons2 + 1, hidden_neurons3)
weights4 = initialize_weights(hidden_neurons3 + 1, output_neurons)
# ..the weights matrix need to have 1 column more
# because the bias is going to be inserted later on..

errors = np.zeros(training_samples)
errors_mean = np.zeros(epocs)

In [262]:
# ------------- * TRAINING * -------------

# ..for each sample in each epoc..
for i in range(epocs):
    for j in range(training_samples):
        
        # ..inserting the bias into the inputs
        # and multiplying it with it's respective
        # weight matrix..
        input_biased = np.hstack((bias, training_features[j]))
        
        output1 = np.tanh(weights1.dot(input_biased))
        output1_biased = np.hstack((bias, output1))
        
        output2 = np.tanh(weights2.dot(output1_biased))
        output2_biased = np.hstack((bias, output2))
        
        output3 = np.tanh(weights3.dot(output2_biased))
        output3_biased = np.hstack((bias, output3))
        
        result = np.tanh(weights4.dot(output3_biased))
        
        # ..get the error and make it quadractic so it's more noticeble,
        # bigger errors tend to outstand more..
        error = training_labels[j] - result
        errors[j] = (error ** 2)/2
        
        # ..calculates the delta for the layer
        # and propagates it for the layer behind..  
        delta4 = error * (1 - result ** 2)  
        
        vdelta3 = weights4.transpose().dot(delta4) 
        delta3 = vdelta3 * (1 - output3_biased ** 2)
        
        vdelta2 = weights3.transpose().dot(delta3[1:]) # ..skipping the bias..
        delta2 = vdelta2 * (1 - output2_biased ** 2)
        
        vdelta1 = weights2.transpose().dot(delta2[1:]) # ..skipping the bias..
        delta1 = vdelta1 * (1 - output1_biased ** 2)
        
        # ..adjust the weight's matrix for the next sample..
        weights1 += learning_rate * np.outer(delta1[1:], input_biased)
        weights2 += learning_rate * np.outer(delta2[1:], output1_biased)
        weights3 += learning_rate * np.outer(delta3[1:], output2_biased)
        weights4 += learning_rate * np.outer(delta4, output3_biased)
        
    # ..saving the weights to use it
    # later on..
    save_weights("weights_tanh")
            
    # ..stopping when reaching such accuracy in 
    # testing to avoid overfitting..
    if(early_stopping()):
        print("early stopping: model hit 80% or more of accuracy in testing.")
        break
    
    # ..stopping when the model has reached it's 
    # convergence state so it won't start losing
    # performance..
    # if(reached_convergence()):
    #     print("the model stopped learning because it hit it's convergency point.")
    #     break
    
    errors_mean[i] = errors.mean()
    print(f"mean error of epoch ({i}): {errors_mean[i]}")

C:\Users\Yasmim da Cunha\AppData\Local\Temp\ipykernel_46136\335276801.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  errors[j] = (error ** 2)/2
C:\Users\Yasmim da Cunha\AppData\Local\Temp\ipykernel_46136\2826866135.py:61: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  testing_errors[k] = (error * error)/2


mean error of epoch (0): 0.10925650423263214
mean error of epoch (1): 0.1090736224448454
mean error of epoch (2): 0.10889982233210647
mean error of epoch (3): 0.1088355008962699
mean error of epoch (4): 0.108698342142699
mean error of epoch (5): 0.10863312257912194
mean error of epoch (6): 0.10851433488031668
mean error of epoch (7): 0.10847390190284806
mean error of epoch (8): 0.10846106865983253
mean error of epoch (9): 0.10839619864865821
mean error of epoch (10): 0.10834802096519591
mean error of epoch (11): 0.10823784824188712
mean error of epoch (12): 0.10816881187648085
mean error of epoch (13): 0.10814569348686771
mean error of epoch (14): 0.10811728273013575
mean error of epoch (15): 0.10806947222709744
mean error of epoch (16): 0.10803186502774667
mean error of epoch (17): 0.1079893723855598
mean error of epoch (18): 0.10797980432731782
mean error of epoch (19): 0.1079372084162093
mean error of epoch (20): 0.10788996635876477


KeyboardInterrupt: 

In [229]:
# print(load_weights("weights_tanh.npy"))
print(test_model())
# weights1, weights2, weights3, weights4 = load_weights('weights_tanh.npy')


# sample = random.randrange(0, training_samples)
    
# input_biased = np.hstack((bias, training_features[sample]))
    
# output1 = np.tanh(weights1.dot(input_biased))
# output1_biased = np.hstack((bias, output1))

# output2 = np.tanh(weights2.dot(output1_biased))
# output2_biased = np.hstack((bias, output2))

# output3 = np.tanh(weights3.dot(output2_biased))
# output3_biased = np.hstack((bias, output3))

# result = np.tanh(weights4.dot(output3_biased))

# print(result, training_labels[sample])

(array([-0.17560132]), np.int64(0))
